In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split


file_path = 'FS8_PPGUID.csv' 
df = pd.read_csv(file_path, low_memory=False)
df.drop('Unnamed: 24',axis=1, inplace=True)
df['TargetFilename'] = df['TargetFilename'].apply(lambda x: 0 if pd.isnull(x) else len(str(x)))
from sklearn.preprocessing import LabelEncoder
df['LogonId'] = df['LogonId'].fillna(-1)
df['TerminalSessionId'] = df['TerminalSessionId'].fillna(-1)

label_encoder_logon = LabelEncoder()
label_encoder_terminal = LabelEncoder()

df['LogonId'] = df['LogonId'].apply(lambda x: x if x == -1 else label_encoder_logon.fit_transform([x])[0])
df['TerminalSessionId'] = df['TerminalSessionId'].apply(lambda x: x if x == -1 else label_encoder_terminal.fit_transform([x])[0])

df['LogonId'] = df['LogonId'].astype(int)
df['TerminalSessionId'] = df['TerminalSessionId'].astype(int)
df_new = df
df_new = df_new.drop_duplicates()
#df_new.info()
df_new.replace('-', 'null', inplace=True)
#df_new.info()
df_new['CurrentDirectory'] = df_new['CurrentDirectory'].apply(lambda x: -1 if pd.isna(x) or x == 'null' else len(str(x)))
df_new['LogonGuid'] = df_new['LogonGuid'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)


df_new['IntegrityLevel'] = df_new['LogonGuid'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['ParentUser'] = df_new['ParentUser'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['Protocol'] = df_new['Protocol'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['Initiated'] = df_new['Initiated'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['SourceIsIpv6'] = df_new['SourceIsIpv6'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['SourceIp'] = df_new['SourceIp'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['DestinationIsIpv6'] = df_new['DestinationIsIpv6'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['DestinationIp'] = df_new['DestinationIp'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['ParentProcessGuid'] = df_new['ParentProcessGuid'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['SourcePort'] = df_new['SourcePort'].apply(lambda x: -1 if pd.isna(x) or x == 'null' else x)
df_new['DestinationPort'] = df_new['DestinationPort'].apply(lambda x: -1 if pd.isna(x) or x == 'null' else x)
df_new['User'] = df_new['User'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
#df_new.info()
df_new['label'].value_counts()
df_final = df_new
X = df_final.drop(columns=['label'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_benign = X_scaled[df_new['label'] == 0]
X_malicious = X_scaled[df_new['label'] == 1]

X_train_benign, X_test_benign = train_test_split(X_benign, test_size=0.2, random_state=42)


one_class_svm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.1)  
one_class_svm.fit(X_train_benign)


#X_pca_full = pca.transform(X_scaled)


y_test_combined = df_new['label']  


y_pred_svm = one_class_svm.predict(X_scaled)
y_pred_svm = [1 if pred == -1 else 0 for pred in y_pred_svm]  


print("Confusion Matrix (One-Class SVM):\n", confusion_matrix(y_test_combined, y_pred_svm))
print("Classification Report (One-Class SVM):\n", classification_report(y_test_combined, y_pred_svm))

C:\Users\User\AppData\Local\Temp\ipykernel_8832\3915374322.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.replace('-', 'null', inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_8832\3915374322.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['CurrentDirectory'] = df_new['CurrentDirectory'].apply(lambda x: -1 if pd.isna(x) or x == 'null' else len(str(x)))
C:\Users\User\AppData\Local\Temp\ipykernel_8832\3915374322.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

Confusion Matrix (One-Class SVM):
 [[256366  28560]
 [ 53020  10093]]
Classification Report (One-Class SVM):
               precision    recall  f1-score   support

           0       0.83      0.90      0.86    284926
           1       0.26      0.16      0.20     63113

    accuracy                           0.77    348039
   macro avg       0.54      0.53      0.53    348039
weighted avg       0.73      0.77      0.74    348039



In [5]:
import numpy as np

X_test_malicious_sampled = X_malicious[:20000]

X_test_combined = np.concatenate([X_test_benign, X_test_malicious_sampled], axis=0)

y_test_combined = np.array([0] * len(X_test_benign) + [1] * len(X_test_malicious_sampled))
print("y_test_combined labels:\n", y_test_combined)

print("First 20 labels of y_test_combined:\n", y_test_combined[:20])

unique, counts = np.unique(y_test_combined, return_counts=True)
print("Label counts using np.unique:")
print(dict(zip(unique, counts)))

from collections import Counter
label_counts = Counter(y_test_combined)
print("Label counts using Counter:")
print(label_counts)

y_test_combined labels:
 [0 0 0 ... 1 1 1]
First 20 labels of y_test_combined:
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label counts using np.unique:
{0: 56986, 1: 20000}
Label counts using Counter:
Counter({0: 56986, 1: 20000})


In [7]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix, classification_report

svm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.25)  

svm.fit(X_train_benign)

y_pred_svm = svm.predict(X_test_combined)  

y_pred_svm = np.array([1 if pred == -1 else 0 for pred in y_pred_svm])

print("Confusion Matrix (One-Class SVM):\n", confusion_matrix(y_test_combined, y_pred_svm))
print("Classification Report (One-Class SVM):\n", classification_report(y_test_combined, y_pred_svm))



Confusion Matrix (One-Class SVM):
 [[42712 14274]
 [12706  7294]]
Classification Report (One-Class SVM):
               precision    recall  f1-score   support

           0       0.77      0.75      0.76     56986
           1       0.34      0.36      0.35     20000

    accuracy                           0.65     76986
   macro avg       0.55      0.56      0.56     76986
weighted avg       0.66      0.65      0.65     76986

